In [88]:
from selenium import webdriver
# from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options
import time
# options = Options()
import pandas as pd
import random
user_agent_list = [
'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36'
]
chrome_options = Options()  
chrome_options.add_argument('--window-size=1920,1080')
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36")
chrome_options.add_argument(f"{random.choice(user_agent_list)}")
chrome_options.add_argument("--enable-javascript")
chrome_options.add_argument('start-maximized')
chrome_options.add_argument("ignore-certificate-errors")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
###
chrome_options.add_argument("enable-automation")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--dns-prefetch-disable")


def create_chrome_driver():
	driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
	# driver = webdriver.Firefox(executable_path='/Users/pa20316035/Documents/geckodriver')
	u_a= driver.execute_script("return navigator.userAgent")
	print("User agent:",u_a)
	return driver


In [89]:
brand_list = ['samsung','huawei','apple','xiaomi','mi notebook']
def get_products(url, country_name,product_type,driver):
    products=[]
    links=[]
    country = []
    p_type = []
    try:
        #print('... ',product_type)
        driver.get(url)
        try:
            time.sleep(3)
            cookies_xpath = 'sp-cc-accept'#'//input[@id="sp-cc-accept"]'
            cookies_btn = WebDriverWait(driver,5).until(EC.visibility_of_element_located((By.ID,cookies_xpath)))#driver.find_element(By.XPATH, cookies_xpath)
            cookies_btn.click()
        except Exception as e:
            pass
            
        next_xpath='//li[@class="a-last"]//a'
        next_new_xpath='//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]'
        #next_new_xpath='//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]'
        last_xpath='//div[@class="a-text-center"]'
        last_xpath_new='//div[@class="a-section a-spacing-large a-spacing-top-large a-text-center s-pagination-container"]'    

        lastpage_text_val=''
        try:
            lastpage_xpath='//span[@class="s-pagination-strip"]'
            pagination_text=WebDriverWait(driver,60).until(EC.visibility_of_element_located((By.XPATH,lastpage_xpath)))
            lastpage_text_val = pagination_text.text
            #print('lastpage text : ',lastpage_text_val)
        except Exception as e:
            try:
                pagination_text=WebDriverWait(driver,60).until(EC.visibility_of_element_located((By.XPATH,last_xpath)))
                lastpage_text_val = pagination_text.text
                #print('lastpage text : ',lastpage_text_val)
            except Exception as e:
                lastpage_text_val = '1'
                #print('unable to find lastpage number')
        
        if '\n' in lastpage_text_val:
            lastpage_text_val = lastpage_text_val.replace('Zurück','')
            last_page_number=int(re.findall(r'(\d+)\s*\n(?:Next|Weiter)',lastpage_text_val)[0])
        else:
            if lastpage_text_val == '1':
                last_page_number = 1
            elif '.' in lastpage_text_val:
                last_page_number=int(re.findall(r'\.\s*(\d+)\s*(?:Next|Weiter)',lastpage_text_val)[0])
            else:
                last_page_number=int(re.findall(r'\s*(\d)\s*Next',lastpage_text_val)[0])
        #print("...  %s of pages to be scraped" %last_page_number)
        
        #last_page_number = 1 ###################################################### COMMENT THIS LINE
        
        for i in range(1,last_page_number+1):
            matchObj = re.search('page\=\d+',url)
            newpage = 'page='+str(i)
            if matchObj != None:
                url = re.sub(r'page\=\d+',newpage,url)
            else:
                url = url+'&'+newpage
            my_url=url
            #print(my_url)
            if i==1:
                if country_name == 'United States':
                    driver.get(my_url)
                driver.get(my_url)
            else:
                pass
            driver.get(my_url)
            time.sleep(3)
            prod_xpath='//div[@data-component-type="s-search-result"]//h2//a'  #have to remove h2
            detail_xpath='//div[@data-component-type="s-search-result"]'        
            prod_names=WebDriverWait(driver,30).until(EC.visibility_of_all_elements_located((By.XPATH,prod_xpath)))
            details=WebDriverWait(driver,1).until(EC.visibility_of_all_elements_located((By.XPATH,detail_xpath)))
            for prod, detail in zip(prod_names, details):
                my_link=prod.get_attribute('href')
                links.append(my_link)
                country.append(country_name)
                p_type.append(product_type)
            #print("....... Page %s is done" %i) 
    except Exception as e:
        print(e)
        pass
    df=pd.DataFrame(list(zip(links,country,p_type)),columns=["Product_Link",'Country_Name','Product_Type'])
    return df

In [90]:
import re
from datetime import datetime

def extract_product_name(data):
    name = data.split("(")[0].strip()
    storage = data.split("(")[1].split(")")[0]
    return name, storage
    
def extract_date(string):
    # Define the patterns to match the date formats
    patterns = [
        r"on\s+(\w+\s+\d+\s*,\s*\d{4})",             # Format: "on January 16, 2023"
        r"on\s(\d+\s+\w+\s\d{4})"            # Format: "on 6 May 2023"
    ]

    for pattern in patterns:
        # Find the first occurrence of the pattern in the string
        match = re.search(pattern, string)
        if match:
            # Extract the matched date string
            date_str = match.group(1)
            
            try:
                # Convert the date string into a datetime object
                date = datetime.strptime(date_str, "%d %B %Y").date()
            except Exception as e:
                try:
                    # Convert the date string into a datetime object (for the second format)
                    date_str = date_str.replace(',','')
                    date = datetime.strptime(date_str, "%B %d %Y").date()
                except Exception as e:
                    print(e)
                    pass

    return date

In [91]:
final_df = pd.DataFrame(columns = ["Prodcut_Reviwer",'Review_Title_Content','Review_Date','Review_Rating','Review_Description'])
def get_review(driver,product_title):
    prodcut_review_path = "//div[@class='a-section review aok-relative']"
    global final_df
    global prod_review
    prodcut_reviwer_list = []
    review_title_content_list = []
    review_date_list = []
    review_rating_list = []
    review_description_list = []
    try:
        prod_review=WebDriverWait(driver,30).until(EC.visibility_of_all_elements_located((By.XPATH,prodcut_review_path)))
    except Exception as e:
        print('====',e)
    for r in prod_review:    
        try:
            prodcut_reviwer=r.find_elements_by_class_name("a-profile-name")
            prodcut_reviwer_list.append(prodcut_reviwer[0].text)

            review_title_content=r.find_elements_by_class_name("review-title-content")
            review_title_content_list.append(review_title_content[0].text)

            review_date = extract_date(r.text)
            print("date================",review_date)
            review_date_list.append(review_date)

            element = r.find_element_by_class_name("a-link-normal")
            #print(element.text)
            review_rating = element.get_attribute("title")
            review_rating_list.append(review_rating)


            review_description=r.find_elements_by_class_name("review-text-content") 
            review_description_list.append(review_description[0].text)

        except Exception as e:
            print('---->',e)

    df=pd.DataFrame(list(zip(prodcut_reviwer_list,review_title_content_list,review_date_list,review_rating_list,review_description_list)),
                    columns=["Prodcut_Reviwer",'Review_Title_Content','Review_Date','Review_Rating','Review_Description'])
    print(df.shape)
    name, storage = extract_product_name(product_title)
    df['Prodcuts_Name'] = name
    df['Storage'] = storage
    final_df = pd.concat([final_df,df])

In [92]:
def get_product_details(links,driver):
    
    link_length = len(links)
    for count,url in enumerate(links,start=1):
        driver.get(url)
        try:
            prodcuts = []
            page = driver.page_source
            soup = BeautifulSoup(page,'html.parser')
            product_title = soup.find("span", {"id": "productTitle"})
            product_title = product_title.text
            product_title =re.sub(r'^$\n', '', product_title.strip(), flags=re.MULTILINE)
            print(product_title)
            prodcuts.append(product_title)
        except Exception as e:
            print(e)
            product_title=''
        try:
            time.sleep(3)
            review_click = "//*[@id='reviews-medley-footer']/div[2]/a"
            review_btn = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,review_click)))#driver.find_element(By.XPATH, cookies_xpath)
            review_btn.click()
        except Exception as e:
            print(e)
            pass
        prodcut_review_path = "//div[@class='a-section review aok-relative']"
        try:
            prod_review=WebDriverWait(driver,30).until(EC.visibility_of_all_elements_located((By.XPATH,prodcut_review_path)))
        except Exception as e:
            print(e)
            
        for i in range(100):
            time.sleep(4)
            get_review(driver,product_title)
            page_number_xpath = "//*[@id='cm_cr-pagination_bar']/ul/li[2]/a"
            try:
                next_btn = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,page_number_xpath)))#driver.find_element(By.XPATH, cookies_xpath)
                next_btn.click()
            except Exception as e:
                break
                print(e)


In [93]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
import pandas as pd


country_name = 'India'
smartphone_url = 'https://www.amazon.in/s?i=electronics&bbn=1805560031&rh=n%3A976419031%2Cn%3A1389401031%2Cn%3A1389432031%2Cn%3A1805560031%2Cp_89%3AApple&dc&qid=1683721705&rnid=3837712031&ref=sr_nr_p_89_2&ds=v1%3ALkr9phurYQR8XNuQ1qI4%2BZOuJZf5Z24yLjP7lF6c49M'

# smartphone_url = 'https://www.amazon.in/s?i=electronics&bbn=1805560031&rh=n%3A976419031%2Cn%3A1389401031%2Cn%3A1389432031%2Cn%3A1805560031%2Cp_89%3AApple%2Cp_n_size_two_browse-vebin%3A15564000031&dc&qid=1683738649&rnid=15563994031&ref=sr_nr_p_n_size_two_browse-vebin_3&ds=v1%3A%2BSPJV5zsJy6Q7algoqQr3ONT2KFZbEvIvBkck1S6u8Y'

p_type = 'smartphone'
def get_data():
    driver = create_chrome_driver()
    products_df=get_products(smartphone_url, country_name,p_type,driver)
    links = products_df['Product_Link'].to_list()
    get_product_details(links,driver)
    print(links)
    driver.quit()

get_data()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 113.0.5672
[WDM] - Get LATEST chromedriver version for 113.0.5672 google-chrome
[WDM] - Driver [/Users/pa20316035/.wdm/drivers/chromedriver/mac64/113.0.5672.63/chromedriver] found in cache
/var/folders/cl/0_f0l5hj0pd433cfl9h52zsw0000gp/T/ipykernel_66330/2678351688.py:40: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)


User agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15
Message: 

[]


In [86]:
print(final_df.shape)
final_df = final_df.drop_duplicates()
final_df.to_excel('sample_data_apple_new1.xlsx',index=False)

(7106, 7)


In [87]:
final_df.shape

(3792, 7)

In [110]:
import pandas as pd
csv_file = '/Users/pa20316035/Downloads/a070107010-056.csv'
df = pd.read_csv(csv_file,encoding= 'unicode_escape',skiprows=2)
df.head(15)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa8 in position 2: invalid start byte